The next cell downloads the required data set to carry out the workshop. This kind of code works on Google Colab, and if you are using another platform to run the notebook, you might need to manually download the data directory from the GitHub repository and put it in the same path as the notebook file.

In [1]:
!curl https://raw.githubusercontent.com/APSV-UPM/BusinessIntelligence/main/data/data.csv > data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1516k  100 1516k    0     0  3206k      0 --:--:-- --:--:-- --:--:-- 3212k


In this workshop, we will work with a data set representing a logistics process. These data contain events from a logistics process in which a series of goods or items are transported from one station to another by train. It is based on a real-world process, but it has been simplified to make it easier to work with.
Each step in the process is represented by an event, and the events have a timestamp that represents when the event was recorded. The types of events appear in Spanish; the following table is a translation of these into English.

Spanish | English
---|---
'Comienzo de ruta'         | 'START OF ROUTE'
'Descarga de vagones'      | 'END OF WAGON UNLOADING'
'Final carga'              | 'END OF WAGON LOADING'
'Final de ruta'            | 'END OF ROUTE'
'Llegada de vagones'       | 'ARRIVAL OF WAGONS TO DESTINATION'
'Salida de vagones'        | 'DEPARTURE OF WAGONS FROM ORIGIN'

Our goal is to obtain as much information as we can from this data set. We will achieve this in two ways: we will answer questions with numeric values (e.g., how many trains take part in the process?) or we will generate some charts to present information in a visual way (e.g. how is the distribution of items transported throughout the year?)


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
df = pd.read_csv("data.csv")

# Dataset exploration
Any project related to data analysis starts with a study of the data itself.
* What kind of data do we have?
* How is it organized? How many columns does each dataframe have?
* Are there wrong or missing values?
* What does each value of a column mean? How many different values are there in each column?

With the examples we saw in the previous workshop we try to answer those questions.

In [4]:
df.head()

,Unnamed: 0,trip_id,train_name,date,cargo,station_id,event,cargo_type,fuel_level,ended
0,0,93OBAJQT4H,VXRCS,2025-04-02 10:57:44,24.0,station_B,Descarga de vagones,Textiles,1146.000,True
1,1,95DWY7U00J,VXRCS,2024-09-18 14:29:50,0.0,station_E,Descarga de vagones,Machinery,0.000,True
2,2,G2IJESVUP4,UDAXI,2025-06-25 18:00:00,0.0,station_A,Comienzo de ruta,_,590.056,True
3,3,OTSN5U4W9R,VXRCS,2024-09-06 13:16:24,0.0,station_D,Llegada de vagones,Electronics,1031.616,True
4,4,87C7G23CP7,HHEXD,2025-02-06 15:21:28,0.0,station_G,Llegada de vagones,Machinery,179.750,True


In [5]:
df.event.describe()

,event
count,15426
unique,6
top,Salida de vagones
freq,4413


In [6]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15426 entries, 0 to 15425
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  15426 non-null  int64  
 1   trip_id     15426 non-null  object 
 2   train_name  15426 non-null  object 
 3   date        15426 non-null  object 
 4   cargo       15415 non-null  float64
 5   station_id  15426 non-null  object 
 6   event       15426 non-null  object 
 7   cargo_type  15426 non-null  object 
 8   fuel_level  15417 non-null  float64
 9   ended       15426 non-null  bool   
dtypes: bool(1), float64(2), int64(1), object(6)
memory usage: 1.1+ MB


In [7]:
df.describe()

,Unnamed: 0,cargo,fuel_level
count,15426.000000,15415.000000,15417.000000
mean,7712.500000,13.521894,606.365736
std,4453.246961,20.306530,331.874793
min,0.000000,-50.000000,-195.000000
25%,3856.250000,0.000000,353.672000
50%,7712.500000,3.000000,652.960000
75%,11568.750000,19.000000,863.616000
max,15425.000000,100.000000,8731.000000


In [ ]:
# Use as many cells as you need for your exploration

# Data Cleaning
After a first view of the data and before starting to work with it, we must clean it. This process is called preprocessing, and it is crucial to be able to obtain good results. We will discard invalid data, fill missing values, drop redundant information, correct typos, etc. We need to create a data set with the following restrictions.

* All columns must contain relevant information. If a column does not contain relevant information, it should be removed using the method `data.drop(columns = [column_name])` (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)
* Duplicate rows should be removed using the method `data.drop_duplicates()` (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)
* Rows with future dates (after September 30, 2025) should be eliminated
* All fields should have a valid value.
* String columns cannot be empty or null
* Numeric columns should be in the correct ranges. The maximum cargo is 200 and the maximum fuel 2000.
* Rows with null or incorrect values should be discarded (https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html). This includes rows with illegal values according to the dataset description (e.g. a negative number of items).
* The types of the columns must correspond to the type of data they contain (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html)

Python and Pandas have a multitude of methods that make it easier to work with dates. In this workshop, it can be useful to use methods that allow us to obtain certain fragments of a date (hour of day, the day of the week, etc.) https://docs.python.org/3/library/datetime.html. An effective way to apply a change to an entire column is with a syntax like `df.date.dt.hour`, which allows us to get the hour of the dates in the column `date`

In [10]:
# Convert dates
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Drop irrelevant columns (in your case: 'Unnamed: 0')
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# Drop duplicates
df = df.drop_duplicates()

# Remove future dates
df = df[df['date'] <= '2025-09-30']

# Remove nulls or empty strings
df = df.dropna()
df = df[df['event'].str.strip() != '']

# Filter by valid numeric ranges
df = df[(df['cargo'].between(0, 200)) & (df['fuel_level'].between(0, 2000))]

# Fix types
df = df.astype({'cargo': float, 'fuel_level': float, 'event': str})

# Show data
print(df.info())
print(df.describe(include='all'))
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 15372 entries, 0 to 15425
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   trip_id     15372 non-null  object        
 1   train_name  15372 non-null  object        
 2   date        15372 non-null  datetime64[ns]
 3   cargo       15372 non-null  float64       
 4   station_id  15372 non-null  object        
 5   event       15372 non-null  object        
 6   cargo_type  15372 non-null  object        
 7   fuel_level  15372 non-null  float64       
 8   ended       15372 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(5)
memory usage: 1.1+ MB
None
           trip_id train_name                           date         cargo  \
count        15372      15372                          15372  15372.000000   
unique         732          3                            NaN           NaN   
top     BYY2CONYL1      UDAXI                     

# Data Analytics

Once we know about the data that we are using and have cleaned it, we need to ask what kind of useful information we can extract from those data. It is a good idea to make a kind of brainstorming of possible questions, then take the list of resulting questions and sort them by their difficulty, and finally begin to answer them starting with the easiest ones. For this session, that process has already been done for you, and the questions are already sorted by difficulty. In some cases, the answer to a question can be obtained from a graphic, so use them to your advantage.

Those questions are grouped into 10 blocks. All questions in a block can be solved using the same strategy; e.g., to answer "What was the maximum number of packages transported in a train?" or "What was the minimum number of packages transported in a train?", we will make the same process, but changing the function that we apply to the data. Your task is to answer at least one question in each block. At the end of the workshop session, you will fill out a Moodle questionnaire with 10 questions, one for each block.

## Block 0

- How many records are in the dataset?
- How many columns are in the dataset?

In [17]:

print("Number of records:", df.shape[0])

print("Number of columns:", df.shape[1])


Number of records: 15372
Number of records: 9


## Block 1

- How many trains are in the dataset?
- How many stations are in the dataset?
- How many trips are in the dataset?
- How many different cargo types are in the dataset?

In [18]:
# Number of trains
num_trains = df['train_name'].nunique()
print("Number of trains:", num_trains)

# Number of stations
num_stations = df['station_id'].nunique()
print("Number of stations:", num_stations)

# Number of trips
num_trips = df['trip_id'].nunique()
print("Number of trips:", num_trips)

# Number of cargos
num_cargo_types = df['cargo_type'].nunique()
print("Number of cargo types:", num_cargo_types)


Number of trains: 3
Number of stations: 7
Number of trips: 732
Number of cargo types: 6


## Block 2

- How many packages did train UDAXI transport?
- How many packages of Chemicals were transported?
- What is the maximum cargo carried by train UDAXI?

In [20]:
# Write your code here

cargo_udaxi = df.loc[df["train_name"] == "UDAXI", "cargo"].sum()
print("Total cargo transported by UDAXI:", round(cargo_udaxi, 2))

cargo_chemicals = df.loc[df["cargo_type"].str.lower() == "chemicals", "cargo"].sum()
print("Total cargo transported for Chemicals:", round(cargo_chemicals, 2))

max_cargo_udaxi = df.loc[df["train_name"] == "UDAXI", "cargo"].max()
print("Maximum cargo carried by UDAXI:", round(max_cargo_udaxi, 2))

Total cargo transported by UDAXI: 135660.0
Total cargo transported for Chemicals: 41409.0
Maximum cargo carried by UDAXI: 100.0


## Block 3
- Which cargo type was transported the most?
- Which train made more trips?
- Which train transported the most packages?

In [24]:
# Write your code here

top_cargo_type = (df.groupby("cargo_type")["cargo"].sum().sort_values(ascending=False).head(1))
print("Most transported cargo type:", top_cargo_type.index[0])

top_train_trips = df.groupby("train_name")["trip_id"].count().sort_values(ascending=False).head(1)
print("Train with more trips:", top_train_trips.index[0])

top_train_cargo = df.groupby("train_name")["cargo"].sum().sort_values(ascending=False).head(1)
print("Train with most cargo transported:", top_train_cargo.index[0])


Most transported cargo type: Electronics
Train with more trips: UDAXI
Train with most cargo transported: UDAXI


## Block 4
- How many trips were made in January?
- How many packages were transported on Mondays?

In [25]:
# Write your code here
trips_january = df.loc[df["date"].dt.month == 1, "trip_id"].nunique()
print("Number of trips made in January:", trips_january)

cargo_monday = df.loc[df["date"].dt.day_name() == "Monday", "cargo"].sum()
print("Total packages transported on Mondays:", round(cargo_monday, 2))

Number of trips made in January: 51
Total packages transported on Mondays: 39795.0


## Block 5
- Which train has the best fuel efficiency (lowest average consumption)?
- Which train has the worst fuel efficiency (maximum average consumption)?
- What is the average fuel consumption in the whole trip for Textiles?
- How long lasted the shortest trip?
- How long lasted the longest trip?

In [29]:
# Write your code here

# Average fuel per train
avg_fuel_per_train = df.groupby("train_name")["fuel_level"].mean()

# Best and worst fuel efficiency
best_fuel_train = avg_fuel_per_train.idxmin()
worst_fuel_train = avg_fuel_per_train.idxmax()

print("Best efficiency:", best_fuel_train, " Avg fuel:", round(avg_fuel_per_train.min(), 2))
print("Worst efficiency:", worst_fuel_train," Avg fuel:", round(avg_fuel_per_train.max(), 2))

# Average fuel for Textiles
avg_fuel_textiles = df.loc[df["cargo_type"].str.lower() == "textiles", "fuel_level"].mean()
print("Avg fuel consumption for Textiles:", round(avg_fuel_textiles, 2))

# Trip durations
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Compute trip durations: difference between first and last event per trip
trip_durations = df.groupby('trip_id')['date'].agg(['min', 'max'])
trip_durations['duration'] = trip_durations['max'] - trip_durations['min']

# Identify shortest and longest trips
shortest_trip = trip_durations['duration'].min()
longest_trip = trip_durations['duration'].max()

print("Shortest trip duration:", shortest_trip)
print("Longest trip duration:", longest_trip)


Best efficiency: HHEXD  Avg fuel: 539.33
Worst efficiency: VXRCS  Avg fuel: 734.74
Avg fuel consumption for Textiles: 670.99
Shortest trip duration: 0 days 03:15:42
Longest trip duration: 0 days 08:54:11
